In [24]:
import pandas as pd
import os
import re
from tqdm import notebook
import spacy
import ast

In [12]:
#!python -m spacy download nl_core_news_sm
nlp = spacy.load('nl_core_news_sm')

In [44]:
notebook.tqdm.pandas()

D:\Anaconda\lib\site-packages\tqdm\std.py:651: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
df = pd.read_csv('../clean_data.csv')

In [33]:
test

['Trijntje Lansink',
 'Martinus Ciprianus',
 'Simon van Zuijderwijk',
 'Margareta van Zuijdewijk']

In [37]:
doc = nlp(df.text[0])

for ent in doc.ents:
    if ent.label_ == 'PER' and ' ' in ent.text:
        print(ent.text, ent.label_)

Martinus Ciprianus PER
Margareta van Zuijderwijk Egtelieden PER
Maert 1741tn den Name Godes PER
Mantinus Cijpriauus PER
Testamenten Codicille PER
Actien Coediten PER
Moeder Trijntje PER
Simon van Zuijderwijk PER
Kind off Kinderen PER
Soo ter Laaster PER
Sterffhuijs Soude PER
Alle ' PER
E. Hendrik PER
Schilip van Soest PER
Mansenun Cijmanunj Margreta PER
Hendenk Boouwes Jclip PER


In [28]:
test = []

for x in ast.literal_eval(df.namen[0]):
    if x['tussenvoegsel'] != None:
        test.append(x['voornaam'] + " " + x['tussenvoegsel'] + " " + x['achternaam'])
    else:
        test.append(x['voornaam'] + " " + x['achternaam'])

In [34]:
for ent in doc.ents:
    if ent.text in test:
        print(ent.text, ent.start_char, ent.end_char, ent.label_)


Martinus Ciprianus 110 128 PER
Simon van Zuijderwijk 2011 2032 PER


In [68]:
def compare(ner, true):
    c = 0
    for x in ner:
        if x in true:
            c += 1
    return c / len(true)
    

In [69]:
def spacy_test(text):
    doc = nlp(text)
    holder = []
    for ent in doc.ents:
        if ent.label_ == 'PER' and ' ' in ent.text:
            holder.append(ent.text)
    return holder
    

In [70]:
def true_name_getter(true):
    holder = []
    for x in ast.literal_eval(true):
        if x['tussenvoegsel'] != None:
            holder.append(x['voornaam'] + " " + x['tussenvoegsel'] + " " + x['achternaam'])
        elif x['voornaam'] and x['achternaam'] != None:
            holder.append(x['voornaam'] + " " + x['achternaam'])
    return holder

In [71]:
def NER_test(row):
    ner = spacy_test(row.text)
    true = true_name_getter(row.namen)
    return compare(ner, true)

In [74]:
ner_result = df.progress_apply(NER_test, axis=1)

In [75]:
ner_result.mean()

0.3359746833682815